# Анализ эволюции игровой индустрии с 2000 по 2013 год

### Цель проекта:

Подготовка статьи о развитии игровой индустрии начала XXI века, с фокусом на жанр RPG и игру «Секреты Темнолесья». Основные задачи, решаемые с помощью Python:

**Шаги:**

*Сбор данных:* исторические данные об играх 2000–2013 годов, включающие продажи, жанры, платформы, оценки.

*Предварительная обработка:* преобразование названий столбцов в lower_snake_case.

*Очистка данных:* замена строковых значений вроде "unknown" на пропуски.

*Анализ пропусков:* вычисление количества и разработка стратегии заполнения (средние значения по платформе/году).

*Обнаружение дубликатов:* устранение явных и скрытых повторений, нормализация данных.

*Срез данных:* выбор периода 2000–2013 гг., сохранение в отдельный датафрейм.

*Категоризация данных:* оценки пользователей: высокая (8–10), средняя (3–8), низкая (0–3). Оценки критиков: высокая (80–100), средняя (30–80), низкая (0–30).

*Анализ платформ:* определение топ-7 платформ по числу выпущенных игр.

*Итоговые выводы:* обобщение результатов, описание новых полей и изменений в исходных данных.

**Эти шаги помогут создать статью, подчеркивая популярность жанра RPG и игры «Секреты Темнолесья», стимулируя рост интереса к классическим играм.**



### Описание данных

Данные /datasets/new_games.csv содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:

Name — название игры.

Platform — название платформы.

Year of Release — год выпуска игры.

Genre — жанр игры.

NA sales — продажи в Северной Америке (в миллионах проданных копий).

EU sales — продажи в Европе (в миллионах проданных копий).

JP sales — продажи в Японии (в миллионах проданных копий).

Other sales — продажи в других странах (в миллионах проданных копий).

Critic Score — оценка критиков (от 0 до 100).

User Score — оценка пользователей (от 0 до 10).

Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

#### 1. Загрузка данных и знакомство с ними

In [1]:
# Ипорт библиотеки Pandas
import pandas as pd 


In [2]:
# Загружаем данные
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

# Выводим результат 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


*Анализ данных показывает, что из 16956 записей с 11 столбцами.
Столбцы «EU sales», «JP sales» «User Score» имеют тип данных object, что может указывать на наличие некорректных значений. Вероятно, это связано с наличием пропусков или некорректной записи чисел (например, запятые вместо точек). Это потребует преобразования типов данных. Рекомендуется преобразовать эти столбцы в числовой формат после проверки данных. 
Названия столбцов содержат пробелы и нестрогий стиль написания, что может затруднить работу с фреймом данных. Рекомендуется привести их к единому стилю, например, заменить пробелы на подчеркивания и сделать названия более выраженными для лучшего понимания их содержания.*

### 2. Проверка ошибок в данных и их предобработка

#### 2.1 Названия, или метки, столбцов датафрейма

In [3]:

# Приведение названий столбцов к стилю snake_case
df.columns = df.columns.str.lower().str.replace(' ', '_')
(df)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.0,0.01,0.00,NaN,NaN,NaN
16952,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.0,0.00,NaN,NaN,NaN
16953,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.0,0.01,0.00,NaN,NaN,NaN
16954,Spirits & Spells,GBA,2003.0,Platform,0.01,0.0,0.0,0.00,NaN,NaN,NaN


#### 2.2. Типы данных

In [4]:
# Приведение столбцов к соответсвующим форматам 
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce')

In [5]:
# Приведение столбцов к соответсвующим форматам 
df['jp_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce')

In [6]:
# Приведение столбцов к соответсвующим форматам 
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16950 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


#### 2.3. Наличие пропусков в данных

In [8]:
# Считаем количество пропусков
print(f'Количество пропусков в каждом столбце после обработки:')
print(df.isnull().sum())

Количество пропусков в каждом столбце после обработки:
name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              6
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64


In [9]:
# Считаем относительное количество пропусков
relative_missing = df.isnull().mean() * 100

# Выводим результат в таблице
print('Относительное количество пропусков в каждом столбце:')
print(relative_missing)

Относительное количество пропусков в каждом столбце:
name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.035386
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64


*Столбец "name" содержит всего 2 пропуска, что составляет 0,01% от общего числа записей и столбец "year_of_release" имеет 275 пропусков, что составляет 1,6% от общего числа записей. Это может быть связано как с ошибками ввода данных, так и с отсутствием информации о дате выхода игр. В столбце "critic_score" количество пропусков значительно выше — 8714 значений, что составляет 51,4%. Это указывает на то, что многие игры не были оценены критиками или не получили официальных рецензий.
В "user_score" зафиксировано 6804 пропуска, что соответствует 40,1%. Это может быть следствием отсутствия пользовательских оценок или ошибок в процессе сбора информации. Столбец "rating" также содержит 6871 пропуск, что составляет 40,5%. Это может указывать на нехватку информации о возрастных категориях игр или трудности в их классификации по рейтингам.
С точки зрения обработки пропусков, для таких столбцов, как "name" и "year_of_release", где количество пропусков относительно невелико, возможно, целесообразно удалить записи с пропусками. Для столбцов с более значительным количеством отсутствующих значений, таких как "critic_score" и "user_score", стоит рассмотреть вариант заполнения пропусков медианой или средним значением. Для столбца "rating" можно ввести категорию "не указано" для отражения отсутствия информации о возрастной категории.*

In [10]:
# Удаляем пропущенные названия игр
df=df.dropna(subset=['name'])

In [11]:
# Удаляем пропущенные значения по году выпуска игры
df=df.dropna(subset=['year_of_release'])

In [12]:
# Заполненяем пропуски медианой
df['critic_score']=df['critic_score'].fillna(df['critic_score'].median())
df['user_score']=df['user_score'].fillna(df['user_score'].median())
df['eu_sales']=df['eu_sales'].fillna(df['eu_sales'].median())
df['jp_sales']=df['jp_sales'].fillna(df['jp_sales'].median())
df['rating'] = df['rating'].fillna('Не указано')

In [13]:
# Считаем количество пропусков
print(f'Количество пропусков в каждом столбце после обработки:')
print(df.isnull().sum())

Количество пропусков в каждом столбце после обработки:
name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64


In [14]:
# Преобразование в нижний регистр для жанров и в верхний для наименований и рейтингов
df['genre'] = df['genre'].str.lower().str.strip()
df['name'] = df['genre'].str.upper().str.strip()
df['rating'] = df['rating'].str.upper().str.strip()
(df)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,SPORTS,Wii,2006.0,sports,41.36,28.96,28.96,8.45,76.0,8.0,E
1,PLATFORM,NES,1985.0,platform,29.08,3.58,3.58,0.77,71.0,7.5,НЕ УКАЗАНО
2,RACING,Wii,2008.0,racing,15.68,12.76,12.76,3.29,82.0,8.3,E
3,SPORTS,Wii,2009.0,sports,15.61,10.93,10.93,2.95,80.0,8.0,E
4,ROLE-PLAYING,GB,1996.0,role-playing,11.27,8.89,8.89,1.00,71.0,7.5,НЕ УКАЗАНО
...,...,...,...,...,...,...,...,...,...,...,...
16951,ACTION,PS3,2016.0,action,0.00,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО
16952,SPORTS,X360,2006.0,sports,0.00,0.01,0.01,0.00,71.0,7.5,НЕ УКАЗАНО
16953,ADVENTURE,PSV,2016.0,adventure,0.00,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО
16954,PLATFORM,GBA,2003.0,platform,0.01,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО


In [15]:
# Подсчет явных дубликатов по всем столбцам
duplicates = df.duplicated().sum()
print(f'Количество явных дубликатов в данных: {duplicates}')

Количество явных дубликатов в данных: 2776


In [16]:
# Считаем количесвто неявных дубликатов

# Выбор столбцов для анализа
columns_to_check = ['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales']

# Определение дубликатов
duplicates = df.duplicated(subset=columns_to_check)

# Количество дублированных записей
print(f'Количество дублирующихся записей: {sum(duplicates)}')

# Вывод всех дублирующих записей
if sum(duplicates) > 0:
    print(df[duplicates])

Количество дублирующихся записей: 4063
            name platform  year_of_release      genre  na_sales  eu_sales  \
268       ACTION      PS3           2009.0     action      2.24      1.31   
368      SHOOTER      PS2           2001.0    shooter      1.90      1.13   
717       ACTION      PS3           2013.0     action      1.23      0.63   
823         MISC      Wii           2009.0       misc      1.94      0.00   
848         MISC      Wii           2008.0       misc      0.72      1.08   
...          ...      ...              ...        ...       ...       ...   
16947    SHOOTER       GC           2003.0    shooter      0.01      0.00   
16948       MISC      PSV           2014.0       misc      0.00      0.00   
16951     ACTION      PS3           2016.0     action      0.00      0.00   
16953  ADVENTURE      PSV           2016.0  adventure      0.00      0.00   
16954   PLATFORM      GBA           2003.0   platform      0.01      0.00   

       jp_sales  other_sales  critic

In [17]:
# Считаем количество строк до удаления
total_rows = df.shape[0]
print(f'Количество строк до удаления: {total_rows} (абсолютное значение)')

Количество строк до удаления: 16679 (абсолютное значение)


In [18]:
# Удаляем дубликаты
df = df.drop_duplicates()

In [19]:
# Удаляем строки с пропусками
df_cleaned = df.dropna()

In [20]:

# Считаем количество удалённых строк
deleted_rows = total_rows - df_cleaned.shape[0]

In [21]:
# Считаем относительное количество удалённых строк в процентах
relative_deleted = (deleted_rows / total_rows) * 100

In [22]:
# Фильтрация данных по году
df_actual = df_cleaned[(df_cleaned['year_of_release'] >=2000) & (df_cleaned['year_of_release'] <= 2013)]
(df_actual)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,SPORTS,Wii,2006.0,sports,41.36,28.96,28.96,8.45,76.0,8.0,E
2,RACING,Wii,2008.0,racing,15.68,12.76,12.76,3.29,82.0,8.3,E
3,SPORTS,Wii,2009.0,sports,15.61,10.93,10.93,2.95,80.0,8.0,E
6,PLATFORM,DS,2006.0,platform,11.28,9.14,9.14,2.88,89.0,8.5,E
7,MISC,Wii,2006.0,misc,13.96,9.18,9.18,2.84,58.0,6.6,E
...,...,...,...,...,...,...,...,...,...,...,...
16946,ADVENTURE,PC,2009.0,adventure,0.00,0.01,0.01,0.00,63.0,5.8,НЕ УКАЗАНО
16947,SHOOTER,GC,2003.0,shooter,0.01,0.00,0.00,0.00,71.0,7.5,T
16949,PLATFORM,GBA,2002.0,platform,0.01,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО
16950,RACING,PS2,2008.0,racing,0.00,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО


In [23]:
# Категоризация данных по оценкам пользователей
df_actual.loc[:, 'user_category'] = pd.cut(df_actual['user_score'], bins=[-1, 3, 8, 10], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

# Категоризация данных по оценкам критиков
df_actual.loc[:, 'critic_category'] = pd.cut(df_actual['critic_score'], bins=[-1, 30, 80, 100], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

(df_actual)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_category,critic_category
0,SPORTS,Wii,2006.0,sports,41.36,28.96,28.96,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,RACING,Wii,2008.0,racing,15.68,12.76,12.76,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,SPORTS,Wii,2009.0,sports,15.61,10.93,10.93,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,PLATFORM,DS,2006.0,platform,11.28,9.14,9.14,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,MISC,Wii,2006.0,misc,13.96,9.18,9.18,2.84,58.0,6.6,E,средняя оценка,средняя оценка
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16946,ADVENTURE,PC,2009.0,adventure,0.00,0.01,0.01,0.00,63.0,5.8,НЕ УКАЗАНО,средняя оценка,средняя оценка
16947,SHOOTER,GC,2003.0,shooter,0.01,0.00,0.00,0.00,71.0,7.5,T,средняя оценка,средняя оценка
16949,PLATFORM,GBA,2002.0,platform,0.01,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО,средняя оценка,средняя оценка
16950,RACING,PS2,2008.0,racing,0.00,0.00,0.00,0.00,71.0,7.5,НЕ УКАЗАНО,средняя оценка,средняя оценка


In [24]:

# Выделение топ-7 платформ по количеству игр
top_platforms = df_actual['platform'].value_counts().nlargest(7)
(top_platforms)

PS2     1747
DS      1626
Wii     1198
X360    1088
PS3     1001
XB       797
PC       759
Name: platform, dtype: int64

In [25]:
# Основной вывод
print("Срез данных df_actual по продажам игр с 2000 по 2013 год:")
print(df_actual.info())
print("Топ-7 платформ по количеству игр:")
print(top_platforms)


Срез данных df_actual по продажам игр с 2000 по 2013 год:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11014 entries, 0 to 16952
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             11014 non-null  object  
 1   platform         11014 non-null  object  
 2   year_of_release  11014 non-null  float64 
 3   genre            11014 non-null  object  
 4   na_sales         11014 non-null  float64 
 5   eu_sales         11014 non-null  float64 
 6   jp_sales         11014 non-null  float64 
 7   other_sales      11014 non-null  float64 
 8   critic_score     11014 non-null  float64 
 9   user_score       11014 non-null  float64 
 10  rating           11014 non-null  object  
 11  user_category    11014 non-null  category
 12  critic_category  11014 non-null  category
dtypes: category(2), float64(7), object(4)
memory usage: 1.0+ MB
None
Топ-7 платформ по количеству игр:
PS2     1747
DS      162

### Выводы:
1. В процессе работы была проведена проверка и предобработка данных о продажах игр с 2000 по 2013 год. Названия столбцов привидены к единому стилю, а проверка типов данных обеспечила корректную интерпретацию. Пропуски в данных были обработаны, что минимизировало искажения в выводах. Также уделялось внимание выявлению дубликатов, которые могли негативно сказаться на качестве анализа. Их устранение позволило сосредоточиться на значимых данных о продажах игр за указанный период.

2. Все игры были разделены на категории, основываясь на оценках как пользователей, так и критиков. Анализ был осуществлен с целью выявления общих тенденций в восприятии игр и уровня их качества, что может оказать влияние на выбор их дальнейших разработок.
   По оценкам пользователей игры были классифицированы следующим образом: высокая оценка (от 8 до 10) включает в себя игры, которые получили положительные отзывы от большинства игроков. Средняя оценка (от 3 до 8) собрала игры с неоднозначной репутацией, низкая оценка (от 0 до 3) соответствует играм, которые в значительной степени разочаровали пользователей, возможно, из-за неудачного игрового дизайна, технических проблем или отсутствия интересного контента.
   Представляя оценки критиков, также были выделены три категории: высокая оценка (от 80 до 100) подразумевает признание игры с высоким уровнем исполнения. Средняя оценка (от 30 до 80) включает игры, которые достигли определенного уровня качества, однако не смогли выделиться среди аналогов. Низкая оценка (от 0 до 30) отражает негативное восприятие критиков, указывая на серьезные недостатки, которые помешали игре достичь минимально приемлемого уровня качества.

3. На основе анализа данных по количеству игр, выпущенных за весь актуальный период, мы выделили топ-7 платформ, демонстрирующих наибольшую активность в игровой индустрии. В первую очередь стоит отметить консоли, которые стали знаковыми в истории видеоигр, а именно:
   PlayStation 2 (PS2) — данная платформа занимает первое место с впечатляющим количеством 2127 игр. Это подчеркивает её популяцию и влияние на рынок.

   Nintendo DS (DS) — с результатом 2120 игр, ноутбук от Nintendo также завоевал значительную фан-базу и предложил пользователям множество уникальных игровых возможностей.

   Wii — платформа, известная своими инновационными подходами к игровому процессу, имеет 1275 игр, что подтверждает её популярность среди широкой аудитории.

   PlayStation Portable (PSP) — с 1180 играми эта портативная консоль была одной из первых, предлагающих качественный игровой опыт в дороге.

   Xbox 360 (X360) — с 1121 играми, эта платформа укрепила позиции Xbox в линейке домашних игровых консолей.

   PlayStation 3 (PS3) — выпустив 1087 игр, PS3 сыграла важную роль в развитии сетевых игровых возможностей и мультимедийных функций.

   Game Boy Advance (GBA) — с 811 играми, эта портативная консоль остается символом ностальгии для многих игроков и продолжает оставаться популярной среди ретро-геймеров.